# Actividad 2: PRÁCTICA DE CLASIFICACIÓN DE TEXTOS
## Borja Lacalle Álvarez

In [2]:
#importar librerias para el desarrollo de la actividad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import nltk

In [3]:
# leer dataset spam.csv
spam = pd.read_csv('spam.csv')
spam.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
#normalizar el texto quitando signos de puntuacion y stopwords, entre otras cosas
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

wpt = WordPunctTokenizer()
stop_words = set(stopwords.words('english'))

def normalize_document(doc):
    doc = doc.replace("!", "").replace("¡", "").replace(",", "").replace(".", "").replace(";", "")
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [5]:
norm_spam=[]

for document in spam['text']:
    norm_spam.append(normalize_document(document))

#ver solo los primeros 20 documentos normalizados
norm_spam[:20]


['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 "free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question ( std txt rate ) & c ' apply 08452810075over18 '",
 'u dun say early hor u c already say',
 "nah ' think goes usf lives around though",
 "freemsg hey darling ' 3 week ' word back ' like fun still ? tb ok xxx std chgs send å £ 150 rcv",
 'even brother like speak treat like aids patent',
 "per request ' melle melle ( oru minnaminunginte nurungu vettam )' set callertune callers press * 9 copy friends callertune",
 'winner valued network customer selected receivea å £ 900 prize reward claim call 09061701461 claim code kl341 valid 12 hours',
 'mobile 11 months ? u r entitled update latest colour mobiles camera free call mobile update co free 08002986030',
 "' gonna home soon ' want talk stuff anymore tonight k ? ' cried enough today",
 'six chances win cash 100 20000 pounds txt > c

In [6]:
#convertir norm_spam en una matriz tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(norm_spam)

tfidf_array = tfidf.toarray()
tfidf_array


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
#dividir el dataset en train (80%) y test (20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf, spam['label'], test_size=0.2, random_state=42)



#### Clasificador bayesiano ingenuo (Naive Bayes)

In [8]:
# entenar modelo bayesiano ingenuo
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)


MultinomialNB()

In [10]:
#predecir con el modelo entrenado
y_pred = nb.predict(X_test)
y_pred

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'spam'], dtype='<U4')

In [11]:
#evaluar el modelo con la matriz de confusion
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[965,   0],
       [ 37, 113]])

La estructura típica de una matriz de confusión binaria es la siguiente:

[[Verdaderos Negativos (TN), Falsos Positivos (FP)]  
 [Falsos Negativos (FN), Verdaderos Positivos (TP)]]

Por lo tanto, estos valores tienen el siguiente significado:

TN (True Negatives): La cantidad de mensajes clasificados correctamente como "ham" (no spam).  
FP (False Positives): La cantidad de mensajes que fueron clasificados incorrectamente como "spam" cuando en realidad son "ham".  
FN (False Negatives): La cantidad de mensajes que fueron clasificados incorrectamente como "ham" cuando en realidad son "spam".  
TP (True Positives): La cantidad de mensajes clasificados correctamente como "spam".

